# Project work

In [1]:
import numpy as np
import random
import copy
import matplotlib.pyplot as plt
from tqdm import tqdm

### Utility functions

In [2]:
# Evaluate the difficulty of each problem
def evaluate_problem_difficulty(problems):
    difficulty_scores = []
    
    for i, problem in enumerate(problems):
        X, y, variables = problem["X"], problem["y"], problem["variables"]
        
        num_vars = len(variables)  
        num_samples = X.shape[1]  
        variability = np.std(y)  

        difficulty = (num_vars * 2) + (num_samples / 5000) + (variability / (np.median(y) + 1e-8))
        difficulty_scores.append((i + 1, difficulty, X, y, variables))

    return difficulty_scores

In [6]:
problems = []
for i in range(1,9):
  data = np.load(f"data/problem_{i}.npz")
  X = data['x']
  y = data['y']
  num_vars = X.shape[0]
  variables = [f"x{i}" for i in range(num_vars)]
  problems.append({"X": X, "y": y, "variables": variables})

difficulty_ranking = evaluate_problem_difficulty(problems)

for problem_id, score, X, y, variables in difficulty_ranking:
    print(f"Problem {problem_id}: X = {X.shape}, y = {y.shape}, variables = {variables}, Difficulty Score = {score:.2f}")

Problem 1: X = (1, 500), y = (500,), variables = ['x0'], Difficulty Score = -6.51
Problem 2: X = (3, 5000), y = (5000,), variables = ['x0', 'x1', 'x2'], Difficulty Score = 50.01
Problem 3: X = (3, 5000), y = (5000,), variables = ['x0', 'x1', 'x2'], Difficulty Score = 9.61
Problem 4: X = (2, 5000), y = (5000,), variables = ['x0', 'x1'], Difficulty Score = 8.77
Problem 5: X = (2, 5000), y = (5000,), variables = ['x0', 'x1'], Difficulty Score = 5.23
Problem 6: X = (2, 5000), y = (5000,), variables = ['x0', 'x1'], Difficulty Score = 3.98
Problem 7: X = (2, 5000), y = (5000,), variables = ['x0', 'x1'], Difficulty Score = 14.22
Problem 8: X = (6, 50000), y = (50000,), variables = ['x0', 'x1', 'x2', 'x3', 'x4', 'x5'], Difficulty Score = 6.30


### Node class

In [ ]:
# Node class for expressing trees
class Node:
    """ Represents a single node in the expression tree """
    def __init__(self, data, children=None):
        self.data = data  # Operator, variable, or constant
        self.children = children if children else []
    
    def evaluate(self, variables):
        """ Evaluates the mathematical expression represented by the tree """
        if isinstance(self.data, (int, float)):
            return self.data  # Constant value
        elif isinstance(self.data, str):
            return variables[self.data]  # Variable lookup
        elif callable(self.data):
            try:
                if len(self.children) == 1:
                    return self.data(self.children[0].evaluate(variables))
                elif len(self.children) == 2:
                    return self.data(self.children[0].evaluate(variables),
                                     self.children[1].evaluate(variables))
            except (FloatingPointError, ValueError, ZeroDivisionError):
                return float('inf')  # Penalize invalid expressions

    def to_formula(self):
        """ Returns a human-readable formula representation of the expression tree"""
        if isinstance(self.data, (int, float)):
            return str(round(self.data, 2))
        if isinstance(self.data, str):
            return self.data  # Variable name
        if callable(self.data):
            if len(self.children) == 1:
                return f"{UNARY_SYMBOLS.get(self.data, '?')}({self.children[0].to_formula()})"
            elif len(self.children) == 2:
                return f"({self.children[0].to_formula()} {OP_SYMBOLS.get(self.data, '?')} {self.children[1].to_formula()})"
        return "?"
    
    def extract_variables(self):
        """ Extracts variables used in the expression tree """
        if isinstance(self.data, str):  # If the node is a variable
            return [self.data]
        elif isinstance(self.data, (int, float)):
            return []  # Constants do not contain variables
        else:
            variables = []
            for child in self.children:
                variables.extend(child.extract_variables())
            return variables

### Handling Math Operators

In [ ]:
# Protected mathematical operations
#Prevent division by zero
def protected_div(x, y):
    return np.divide(x, y + 1e-6)  
#Avoid log(0) or negative numbers
def protected_log(x):
    return np.log(np.abs(x) + 1e-6)
#Ensure sqrt of non-negative values
def protected_sqrt(x):
    return np.sqrt(np.abs(x))

# Define available operators
BINARY_OPS = [np.add, np.subtract, np.multiply, protected_div, np.power]
UNARY_OPS = [np.sin, np.cos, np.tan, protected_log, np.exp, protected_sqrt, np.abs]

# Operator symbol mapping for better readable expressions
OP_SYMBOLS = {np.add: '+', np.subtract: '-', np.multiply: '*', protected_div: '/', np.power: '^'}
UNARY_SYMBOLS = {np.sin: 'sin', np.cos: 'cos', np.tan: 'tan', protected_log: 'log',
                 np.exp: 'exp', protected_sqrt: 'sqrt', np.abs: 'abs'}

### Calculate fitness

In [ ]:
#*********************************** Fitness Function
def calculate_fitness(individual, variables, X, y, penalty=4.5):
    try:
        y_pred = np.array([individual.evaluate(dict(zip(variables, x))) for x in X.T])
        if np.any(np.isnan(y_pred)) or np.any(np.isinf(y_pred)):
            return float('inf')  # Penalize invalid expressions
        used_variables = set(individual.extract_variables())
        complexity_penalty = penalty * (len(variables) - len(used_variables)) if len(used_variables) < len(variables) else 0
        return np.mean((y - y_pred) ** 2) + complexity_penalty
    except (FloatingPointError, ValueError, TypeError):
        return float('inf')

### Generate random trees

In [ ]:
def generate_random_tree(max_depth, variables, depth=0, method='grow'):
    """Recursively generates a random expression tree with controlled depth"""
    
    constant_prob = 0.4

    if depth >= max_depth or (depth > 0 and random.random() < 0.2):
        if random.random() < 0.7:
            return Node(random.choice(variables))  #variable node
        else:
            return Node(round(random.uniform(-10, 10), 2))  #random constant node between -10 and 10
    
    # If method is 'full', generate fully-grown trees
    if method == 'full':
        if random.random() < 0.5:
            op = random.choice(UNARY_OPS)
            return Node(op, [generate_random_tree(max_depth, variables, depth + 1, method='full')])
        else:
            op = random.choice(BINARY_OPS)
            if random.random() < constant_prob:
                #Randomly decide which child will be a constant
                if random.random() < 0.5:
                    #Left child is a constant
                    left_child = Node(random.uniform(-10, 10))  
                    right_child = generate_random_tree(max_depth, variables, depth + 1, method='full')  
                else:
                    #Right child is a constant
                    left_child = generate_random_tree(max_depth, variables, depth + 1, method='full')  
                    right_child = Node(random.uniform(-10, 10))  
            else:
                #No constants
                left_child = generate_random_tree(max_depth, variables, depth + 1, method='full')
                right_child = generate_random_tree(max_depth, variables, depth + 1, method='full')

            return Node(op, [left_child, right_child])
    
    # If method is 'grow', generate trees with variable depth
    elif method == 'grow':
        if random.random() < 0.5:
            op = random.choice(UNARY_OPS)
            return Node(op, [generate_random_tree(max_depth, variables, depth + 1, method='grow')])
        else:
            op = random.choice(BINARY_OPS)
            
            if random.random() < constant_prob:
                if random.random() < 0.5:
                    #Left child is a constant
                    left_child = Node(random.uniform(-10, 10))  
                    right_child = generate_random_tree(max_depth, variables, depth + 1, method='grow')  
                else:
                    #Right child is a constant
                    left_child = generate_random_tree(max_depth, variables, depth + 1, method='grow')  
                    right_child = Node(random.uniform(-10, 10))  
            else:
                #No constants
                left_child = generate_random_tree(max_depth, variables, depth + 1, method='grow')
                right_child = generate_random_tree(max_depth, variables, depth + 1, method='grow')

            return Node(op, [left_child, right_child])
        
    # If method is 'half_and_half', mix 'full' and 'grow' methods
    elif method == 'half_and_half':
        if random.random() < 0.6:
            return generate_random_tree(max_depth, variables, depth, method='full')
        else:
            return generate_random_tree(max_depth, variables, depth, method='grow')


### Generate population

In [ ]:
def generate_population(pop_size, max_depth, variables, method='half_and_half'):
    """Generates an initial population of random expression trees."""
    return [generate_random_tree(max_depth, variables, method=method) for _ in range(pop_size)]

### Evolutionary operators

In [ ]:
#*************************************** Mutation
def mutate(node, variables, generation, max_depth=3, mutation_rate=0.3, top=30):
    """ Mutates a node in the tree, adjusting mutation intensity over generations. """
    if generation >= top:
        mutation_rate = max(0.1, mutation_rate * 0.5)  # Reduce mutation in later generations
    else:
        mutation_rate = min(1.0, mutation_rate * 1.2)  # Increase mutation in early generations
    
    if random.random() < mutation_rate:
        if isinstance(node.data, str):
            node.data = random.choice(variables)  # Change variable
        elif isinstance(node.data, (int, float)):
            node.data *= random.uniform(0.5, 1.5)  # Perturb numeric values
        elif callable(node.data):
            node.data = random.choice(UNARY_OPS if len(node.children) == 1 else BINARY_OPS)
    
    if node.children and generation < top and random.random() < mutation_rate:
        if random.random() < 0.5:
            return copy.deepcopy(generate_random_tree(max_depth, variables))  # Replace subtree
    
    for i in range(len(node.children)):
        node.children[i] = mutate(node.children[i], variables, generation, max_depth, mutation_rate, top)
    
    return node


#*************************************** Crossover
def crossover(parent1, parent2):
    """ Swaps a subtree from parent1 with a subtree from parent2. """
    child1, child2 = copy.deepcopy(parent1), copy.deepcopy(parent2)
    node1, node2 = random.choice(child1.children), random.choice(child2.children)
    node1.data, node2.data = node2.data, node1.data
    node1.children, node2.children = node2.children, node1.children
    return child1

### Settings

In [ ]:
penalties = [1.5, 5e12, 5000_000, 4.5, 5.5, 5.5, 5.5, 5000_000]
pop_size = [50, 150, 150, 100, 100, 100, 100, 150]
tournament_size = [5, 10, 10, 10, 10, 8, 10, 10]
epochs = [50, 50, 50, 50, 50, 50, 50, 50]
max_depth = [3, 5, 5, 5, 5, 5, 5, 5]
methods_pop = ['grow', 'full', 'full', 'full', 'full', 'full', 'full', 'full']
cross_probs = [0.7, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75]
mut_probs = [0.2, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25]